In [28]:
import numpy as np
import sys
sys.path.append('../')
from vocos import Vocos,VocosExp,feature_extractors,models,heads
import torch


In [33]:
# Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" 
# to /home/tansongbin/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th

vocosModelMel = Vocos.from_pretrained("charactr/vocos-mel-24khz")

In [34]:
# feature_extractor = feature_extractors.MelSpectrogramFeatures()
# back_bone = models.VocosBackbone()
# head = heads.ISTFTHead()
x=VocosExp(vocosModelMel.feature_extractor,vocosModelMel.backbone,vocosModelMel.head,48000,48000)

In [35]:
len(x.state_dict().keys()),len(vocosModelMel.state_dict().keys())

(412, 83)

83

In [9]:
x = torch.load(r'/home/tansongbin/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th')

In [ ]:
torch.nn.Conv2d()

In [13]:
import torchaudio

y, sr = torchaudio.load('/data3/tansongbin/vocoder_projects/dataset/input/vocos_input2.wav')
if y.size(0) > 1:  # mix to mono
    y = y.mean(dim=0, keepdim=True)
y = torchaudio.functional.resample(y, orig_freq=sr, new_freq=24000)
y_hat = vocosModelMel(y)
torchaudio.save('/data3/tansongbin/vocoder_projects/dataset/output/vocos_output2.wav',y_hat,24000)

In [15]:
vocosModelMel.

<bound method Module.modules of Vocos(
  (feature_extractor): MelSpectrogramFeatures(
    (mel_spec): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (backbone): VocosBackbone(
    (embed): Conv1d(100, 512, kernel_size=(7,), stride=(1,), padding=(3,))
    (norm): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
    (convnext): ModuleList(
      (0): ConvNeXtBlock(
        (dwconv): Conv1d(512, 512, kernel_size=(7,), stride=(1,), padding=(3,), groups=512)
        (norm): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (pwconv1): Linear(in_features=512, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (pwconv2): Linear(in_features=1536, out_features=512, bias=True)
      )
      (1): ConvNeXtBlock(
        (dwconv): Conv1d(512, 512, kernel_size=(7,), stride=(1,), padding=(3,), groups=512)
        (norm): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (pwconv1): Linear(in_features=5

In [ ]:
def anti_wrapping_function(x):
  return torch.abs(x - torch.round(x / (2 * np.pi)) * 2 * np.pi)


def phase_loss(phase_r, phase_g, n_fft, frames):

    MSELoss = torch.nn.MSELoss()

    GD_matrix = torch.triu(torch.ones(n_fft//2+1,n_fft//2+1),diagonal=1)-torch.triu(torch.ones(n_fft//2+1,n_fft//2+1),diagonal=2)-torch.eye(n_fft//2+1)
    GD_matrix = GD_matrix.to(phase_g.device)

    GD_r = torch.matmul(phase_r.permute(0,2,1), GD_matrix)
    GD_g = torch.matmul(phase_g.permute(0,2,1), GD_matrix)

    PTD_matrix = torch.triu(torch.ones(frames,frames),diagonal=1)-torch.triu(torch.ones(frames,frames),diagonal=2)-torch.eye(frames)
    PTD_matrix = PTD_matrix.to(phase_g.device)

    PTD_r = torch.matmul(phase_r, PTD_matrix)
    PTD_g = torch.matmul(phase_g, PTD_matrix)

    IP_loss = torch.mean(anti_wrapping_function(phase_r-phase_g))
    GD_loss = torch.mean(anti_wrapping_function(GD_r-GD_g))
    PTD_loss = torch.mean(anti_wrapping_function(PTD_r-PTD_g))


    return IP_loss, GD_loss, PTD_loss

In [42]:
n_fft=8
tmp1 = torch.triu(torch.ones(n_fft//2+1,n_fft//2+1),diagonal=1)
tmp2 = torch.triu(torch.ones(n_fft//2+1,n_fft//2+1),diagonal=2)
tmp3 = torch.eye(n_fft//2+1)
tmp = torch.triu(torch.ones(n_fft//2+1,n_fft//2+1),diagonal=1)-torch.triu(torch.ones(n_fft//2+1,n_fft//2+1),diagonal=2)-torch.eye(n_fft//2+1)


In [43]:
tmp1,tmp2,tmp3,tmp

(tensor([[0., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1.],
         [0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.]]),
 tensor([[0., 0., 1., 1., 1.],
         [0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]),
 tensor([[1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.]]),
 tensor([[-1.,  1.,  0.,  0.,  0.],
         [ 0., -1.,  1.,  0.,  0.],
         [ 0.,  0., -1.,  1.,  0.],
         [ 0.,  0.,  0., -1.,  1.],
         [ 0.,  0.,  0.,  0., -1.]]))

In [44]:
0.57/0.3,3.28/1.76

(1.9, 1.8636363636363635)

In [48]:
'v1',57/30,'istft',0.077/0.30

('v1', 1.9, 'istft', 0.25666666666666665)

In [75]:
10.5/8.7

1.2068965517241381

In [74]:
16.4/8.7

1.8850574712643677